In [1]:
# DO NOT REMOVE THESE
%load_ext autoreload
%autoreload 2

In [2]:
# DO NOT REMOVE This
%reload_ext autoreload

In [3]:
from src import visualize as viz
from src import model as mdl
from src import custom as cm

# AWS ALS Factor Importation and Convertion

In [4]:
! aws s3 ls

/bin/sh: aws: command not found


In [21]:
import boto3
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from scipy.spatial import distance_matrix

In [4]:
s3 = boto3.resource('s3')
client = boto3.client('s3') 
my_bucket = s3.Bucket('fp-movielens-data')

In [5]:
for obj in my_bucket.objects.all():
    print(os.path.join(obj.bucket_name, obj.key))

fp-movielens-data/_metadata
fp-movielens-data/e-MJ39QHZPM80UH29RR62G779J/FIFP.ipynb
fp-movielens-data/e-MJ39QHZPM80UH29RR62G779J/metadata/metadata.zip
fp-movielens-data/item_factors.csv
fp-movielens-data/item_factors.csv/_SUCCESS
fp-movielens-data/item_factors.csv/part-00000-40db7616-e552-48cd-bb18-9fba706fe5aa-c000.csv
fp-movielens-data/item_factors.csv/part-00001-40db7616-e552-48cd-bb18-9fba706fe5aa-c000.csv
fp-movielens-data/item_factors.csv/part-00002-40db7616-e552-48cd-bb18-9fba706fe5aa-c000.csv
fp-movielens-data/item_factors.csv/part-00003-40db7616-e552-48cd-bb18-9fba706fe5aa-c000.csv
fp-movielens-data/item_factors.csv/part-00004-40db7616-e552-48cd-bb18-9fba706fe5aa-c000.csv
fp-movielens-data/item_factors.csv/part-00005-40db7616-e552-48cd-bb18-9fba706fe5aa-c000.csv
fp-movielens-data/item_factors.csv/part-00006-40db7616-e552-48cd-bb18-9fba706fe5aa-c000.csv
fp-movielens-data/item_factors.csv/part-00007-40db7616-e552-48cd-bb18-9fba706fe5aa-c000.csv
fp-movielens-data/item_factors.csv

## Item Factors

In [6]:
item_factors = []
for i in list(range(0,10)):
    obj = client.get_object(Bucket='fp-movielens-data', Key='item_factors.csv/part-0000{}-40db7616-e552-48cd-bb18-9fba706fe5aa-c000.csv'.format(i))
    factors = pd.read_csv(obj['Body'], header=None)
    print(len(factors))
    item_factors.append(factors)

item_factors_df = pd.concat(item_factors, axis=0, ignore_index=True)

120036
124236
118776
120540
121170
121926
116340
121170
118230
125286


In [7]:
item_factors_df.columns = ['id', 'features']
item_factors_df.head()

,id,features
0,10,-0.255329
1,10,0.135568
2,10,0.424694
3,10,0.135304
4,10,0.058897


In [8]:
rank = item_factors_df.groupby(['id']).agg('count')[0:1]['features'][1]

In [9]:
item_factors_unstacked = unstack(item_factors_df, rank)
item_factors_unstacked.head()

value,1,2,3,4,5,6,7,8,9,10,...,33,34,35,36,37,38,39,40,41,42
id,,,,,,,,,,,,,,,,,,,,,
1,-0.339038,0.133855,0.431191,0.170945,0.060187,0.008598,0.054160,-0.401662,-0.193391,0.355262,...,-0.052306,-0.392383,0.141144,0.529034,0.494720,-0.141302,-0.088752,-0.589613,-0.088734,-0.093626
2,-0.206917,0.183585,0.479785,0.222415,-0.039014,0.109340,0.026195,-0.383216,-0.142015,0.147401,...,-0.106040,-0.213550,0.180178,0.553887,0.478742,-0.033819,0.001490,-0.661385,0.055707,-0.049976
3,-0.207074,0.255722,0.396710,0.260146,-0.004491,0.083058,0.009378,-0.374817,-0.110707,0.119876,...,-0.175207,-0.145357,0.177796,0.572370,0.468172,-0.011677,0.052738,-0.643598,0.087316,-0.037295
4,-0.106216,0.210775,0.450775,0.287757,-0.121598,0.122187,-0.048723,-0.411978,-0.057141,0.138302,...,-0.213138,-0.150968,0.100139,0.528075,0.437014,-0.018255,0.019652,-0.623169,0.139524,0.032842
5,-0.150020,0.249664,0.427565,0.288323,-0.057598,0.054907,-0.013541,-0.382258,-0.067867,0.162254,...,-0.288030,-0.102036,0.111018,0.560844,0.508875,-0.012856,0.088838,-0.613712,0.123424,0.020842


In [10]:
item_factors_unstacked.to_csv('../data/processed/item_factors.csv')

## User Factors

In [11]:
user_factors = []
for i in list(range(0,10)):
    obj = client.get_object(Bucket='fp-movielens-data', Key='user_factors.csv/part-0000{}-59dd1ef1-da71-4926-b18b-5a0d5f059a90-c000.csv'.format(i))
    factors = pd.read_csv(obj['Body'], header=None)
    print(len(factors))
    user_factors.append(factors)

user_factors_df = pd.concat(user_factors, axis=0, ignore_index=True)

1027362
1024464
1024590
1021818
1022112
1020138
1026438
1022574
1022322
1021818


In [12]:
user_factors_df.columns = ['id', 'features']
user_factors_df.head()

,id,features
0,10,-0.352436
1,10,0.078241
2,10,0.438761
3,10,0.123200
4,10,0.041807


In [13]:
user_factors_unstacked = unstack(user_factors_df, rank)
user_factors_unstacked.head()

value,1,2,3,4,5,6,7,8,9,10,...,33,34,35,36,37,38,39,40,41,42
id,,,,,,,,,,,,,,,,,,,,,
1,-0.380470,0.103641,0.310486,0.078450,0.129363,-0.033363,0.030401,-0.360824,-0.213137,0.370407,...,0.022118,-0.480875,0.194733,0.479119,0.328390,-0.123725,-0.188246,-0.525000,-0.133443,-0.167593
2,-0.226508,0.167774,0.536940,0.235142,-0.036765,0.118235,0.024486,-0.449235,-0.145772,0.214047,...,-0.048399,-0.314272,0.170148,0.567598,0.492936,-0.072118,-0.063404,-0.715515,0.038176,-0.056638
3,-0.240955,0.179579,0.439598,0.231143,0.003269,0.110166,0.018843,-0.457156,-0.184155,0.168279,...,-0.037431,-0.375902,0.142993,0.607431,0.525451,-0.006926,-0.018648,-0.715832,0.011471,-0.136575
4,-0.271722,0.041331,0.368844,-0.016227,0.102699,0.047941,0.020815,-0.335555,-0.272367,0.089399,...,0.136988,-0.530468,0.179969,0.527210,0.440110,0.124539,-0.082171,-0.663929,-0.065500,-0.327186
5,-0.368851,0.104798,0.349379,0.013516,0.070510,0.036399,-0.017281,-0.401808,-0.266340,0.203034,...,0.118943,-0.581576,0.247609,0.590982,0.318466,0.053089,-0.191376,-0.671003,-0.065487,-0.266448


In [14]:
user_factors_unstacked.to_csv('../data/processed/user_factors.csv')

In [18]:
scaler = StandardScaler()
user_factors_scaled = scaler.fit_transform(user_factors_unstacked)
user_factors_scaled = pd.DataFrame(user_factors_scaled)
user_factors_scaled.head()

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,-1.188306,-0.423128,-0.865804,-0.461143,1.503241,-1.420149,0.436637,0.537105,-0.388644,1.678028,...,0.147211,-0.610899,0.022109,-0.623824,-0.597647,-1.195553,-0.860215,0.950591,-1.344480,-0.125517
1,0.610881,0.386616,0.886891,1.049740,-0.870483,0.911214,0.325355,-0.699159,0.774798,0.203294,...,-0.290968,0.543098,-0.195558,0.082682,0.540480,-0.689802,0.302174,-0.429075,0.473121,1.031875
2,0.442050,0.535666,0.133487,1.011178,-0.298466,0.787118,0.219210,-0.809915,0.111883,-0.228379,...,-0.222816,0.116207,-0.435975,0.400747,0.765383,-0.050916,0.718896,-0.431378,0.190288,0.198030
3,0.082511,-1.209848,-0.414129,-1.374060,1.122249,-0.169807,0.256308,0.890445,-1.411600,-0.972348,...,0.861000,-0.954414,-0.108612,-0.239817,0.175094,1.237460,0.127439,-0.055505,-0.624907,-1.790280
4,-1.052518,-0.408519,-0.564778,-1.087267,0.662312,-0.347311,-0.460341,-0.035977,-1.307515,0.099425,...,0.748868,-1.308421,0.490244,0.269406,-0.666289,0.537238,-0.889354,-0.106731,-0.624770,-1.156709


In [19]:
user_factors_scaled.to_csv('../data/processed/user_factors_scaled.csv')